In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

# SPARK 

## EJ NUMERO 1

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras). Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable. Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2). Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento. Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0).

In [19]:
Quijote = 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino.'
Dickens = 'Las noches aqui son largas algunas veces, muy largas; pero nada son en comparacion con las inquietas noches y terribles ensueños de aquel tiempo. Su recuerdo me estremece. Grandes, sombríos fantasmas con maliciosos rostros se sentaban en los rincones de mi cuarto,y de noche se inclinaban sobre mi, incitándome á la locura. Me decian en voces atronadoras que el suelo de la antigua casa en que murió el padre de mi padre, estaba aun manchado de su sangre, derramada por su propia mano en el furor de su locura. Me cubria los oidos con las manos, pero me gritaban y me gritaban hasta que el cuarto se estremecia con sus acentos, y por todas partes oia que en la generacion anterior á la suya la locura durmió, pero que su abuelo habia vivido por años con sus manos entre grillos, para evitar que se hiciesen pedazos. Sabia que decian la verdad, lo sabia bien. Lo habia descubierto años hacia , aunque me lo quisieron ocultar.'
Napoleon = 'Nacido Napoleone di Buonaparte (Nabolione o Nabulione en corso), sólo un año después de que Francia comprara la isla de Córcega a la República de Génova.Napoleone, años después, cambió su nombre por el afrancesado Napoléon Bonaparte. El registro más antiguo de este nombre aparece en un informe oficial fechado el 28 de marzo de 1796. Su familia formaba parte de la nobleza local. Su padre, Carlo Buonaparte, abogado, fue nombrado en 1778 representante de Córcega en la corte de Luis XVI, lugar donde permaneció por varios años, por lo que fue su madre, María Letizia Ramolino, la figura fundamental de su niñez'
Agua = 'El agua (del latín aqua) es una sustancia cuya molécula está formada por dos átomos de hidrógeno y uno de oxígeno (H2O). Es esencial para la supervivencia de todas las formas conocidas de vida. El término agua generalmente se refiere a la sustancia en su estado líquido, aunque la misma puede hallarse en su forma sólida llamada hielo, y en su forma gaseosa denominada vapor. El agua cubre el 71% de la superficie de la corteza terrestre.2 Se localiza principalmente en los océanos donde se concentra el 96,5% del agua total, los glaciares y casquetes polares poseen el 1,74%, los depósitos subterráneos (acuíferos), los permafrost y los glaciares continentales suponen el 1,72% y el restante 0,04% se reparte en orden decreciente entre lagos, humedad del suelo, atmósfera, embalses, ríos y seres vivos.'
Politica = 'La política \(del latín politicus y esta del griego antiguo πολιτικός civil, relativo al ordenamiento de la ciudad o los asuntos del ciudadano\) es una rama de la moral que se ocupa de la actividad, en virtud de la cual una sociedad libre, compuesta por hombres libres, resuelve los problemas que le plantea su convivencia colectiva. Es un quehacer ordenado al bien común. Algunos autores presentan al uso legítimo de la fuerza como la característica principal de la política. Siguiendo con esta definición la política es el ejercicio del poder que busca un fin trascendente. Esta promueve la participación ciudadana ya que posee la capacidad de distribuir y ejecutar el poder según sea necesario para promover el bien común.'
Sintaxis = 'El orden de las palabras en una oración (sintaxis), la puntuación y la acentuación pueden variar según la intención del autor. En ciertas oraciones ello dependerá de que se emita una idea u otra diferente; en ocasiones lo opuesto a lo que se pretende expresar. Antes de empezar a escribir es necesario alinear mentalmente las ideas que se quieren trasladar al papel o al monitor o pantalla de la computadora u ordenador. Una vez ordenadas en la mente, es necesario identificar las ideas principales y las secundarias. Se elabora un esquema en el que se escriban en orden, según la importancia de cada una. Es importante el orden de una oración, ya que de lo contrario resultará carente de sentido. Más aún, si de manera lógica no se es capaz de ordenar lo que se quiere decir, el escrito perderá todo interés. Toda redacción necesita coherencia y cohesión textuales. En términos sencillos redactar es poner ideas en orden pero bien escritas.Algunos de los denominados «medios» cuentan con manuales de estilo, o imparten cursos de capacitación en sus «instalaciones». Personal que labora en empresas que no disponen de estas «facilidades» y que desea superarse toma cursos en instituciones públicas o privadas.'

Documentos = sc.parallelize([(1,Quijote),(2,Dickens),(3,Napoleon),(4,Agua),(5,Politica),(6,Sintaxis)])

PuntajesUser1 = [(1,"antiguo",2),(1,"Buonaparte",1),(1,"Francia",1)]
PuntajesUser2 = [(2,"en",-1),(2,"orden",4)]
PuntajesUser3 = [(3,"no",0)]

Puntajes = sc.parallelize([PuntajesUser1,PuntajesUser2,PuntajesUser3])

In [20]:
Palabras = Documentos.flatMap(lambda x: map(lambda y: ((y,x[0]),1),x[1].split())).reduceByKey(lambda x,y: x+y)
Palabras.collect()

[(('empresas', 6), 1),
 (('cubria', 2), 1),
 (('antigua', 2), 1),
 (('restante', 4), 1),
 (('para', 4), 1),
 (('sentido.', 6), 1),
 (('algo', 1), 1),
 (('disponen', 6), 1),
 (('fuerza', 5), 1),
 (('con', 5), 1),
 (('atm\xc3\xb3sfera,', 4), 1),
 (('sus', 6), 1),
 (('busca', 5), 1),
 (('a', 4), 1),
 (('a\xc3\xb1os,', 3), 1),
 (('ha', 1), 1),
 (('poder', 5), 2),
 (('\xc3\xa1', 2), 2),
 (('continentales', 4), 1),
 (('una', 5), 2),
 (('noches,', 1), 1),
 (('actividad,', 5), 1),
 (('manera', 6), 1),
 (('principal', 5), 1),
 (('algunas', 2), 1),
 (('me', 2), 4),
 (('misma', 4), 1),
 (('ciudadana', 5), 1),
 (('secundarias.', 6), 1),
 (('hidr\xc3\xb3geno', 4), 1),
 (('sentaban', 2), 1),
 (('aqui', 2), 1),
 (('su', 2), 4),
 (('ordenadas', 6), 1),
 (('s\xc3\xb3lida', 4), 1),
 (('nada', 2), 1),
 (('di', 3), 1),
 (('sobre', 2), 1),
 (('presentan', 5), 1),
 (('formaba', 3), 1),
 (('suya', 2), 1),
 (('parte', 3), 1),
 (('d\xc3\xadas', 1), 1),
 (('gaseosa', 4), 1),
 (('pero', 6), 1),
 (('la', 5), 10),

In [21]:
Palabras = Palabras.map(lambda x: (x[0][0],(x[0][1],x[1])))
Palabras.collect()

[('empresas', (6, 1)),
 ('cubria', (2, 1)),
 ('antigua', (2, 1)),
 ('restante', (4, 1)),
 ('para', (4, 1)),
 ('sentido.', (6, 1)),
 ('algo', (1, 1)),
 ('disponen', (6, 1)),
 ('fuerza', (5, 1)),
 ('con', (5, 1)),
 ('atm\xc3\xb3sfera,', (4, 1)),
 ('sus', (6, 1)),
 ('busca', (5, 1)),
 ('a', (4, 1)),
 ('a\xc3\xb1os,', (3, 1)),
 ('ha', (1, 1)),
 ('poder', (5, 2)),
 ('\xc3\xa1', (2, 2)),
 ('continentales', (4, 1)),
 ('una', (5, 2)),
 ('noches,', (1, 1)),
 ('actividad,', (5, 1)),
 ('manera', (6, 1)),
 ('principal', (5, 1)),
 ('algunas', (2, 1)),
 ('me', (2, 4)),
 ('misma', (4, 1)),
 ('ciudadana', (5, 1)),
 ('secundarias.', (6, 1)),
 ('hidr\xc3\xb3geno', (4, 1)),
 ('sentaban', (2, 1)),
 ('aqui', (2, 1)),
 ('su', (2, 4)),
 ('ordenadas', (6, 1)),
 ('s\xc3\xb3lida', (4, 1)),
 ('nada', (2, 1)),
 ('di', (3, 1)),
 ('sobre', (2, 1)),
 ('presentan', (5, 1)),
 ('formaba', (3, 1)),
 ('suya', (2, 1)),
 ('parte', (3, 1)),
 ('d\xc3\xadas', (1, 1)),
 ('gaseosa', (4, 1)),
 ('pero', (6, 1)),
 ('la', (5, 10)),

In [23]:
Puntajes.collect()

[[(1, 'antiguo', 2), (1, 'Buonaparte', 1), (1, 'Francia', 1)],
 [(2, 'en', -1), (2, 'orden', 4)],
 [(3, 'no', 0)]]

In [24]:
Puntos = Puntajes.flatMap(lambda x: x).map(lambda x: (x[1],(x[0],x[2])))
Puntos.collect()


[('antiguo', (1, 2)),
 ('Buonaparte', (1, 1)),
 ('Francia', (1, 1)),
 ('en', (2, -1)),
 ('orden', (2, 4)),
 ('no', (3, 0))]

In [44]:
Scores = Palabras.join(Puntos)
Scores.collect()

[('antiguo', ((3, 1), (1, 2))),
 ('antiguo', ((5, 1), (1, 2))),
 ('orden', ((6, 3), (2, 4))),
 ('orden', ((4, 1), (2, 4))),
 ('Francia', ((3, 1), (1, 1))),
 ('Buonaparte', ((3, 1), (1, 1))),
 ('en', ((5, 1), (2, -1))),
 ('en', ((3, 4), (2, -1))),
 ('en', ((1, 1), (2, -1))),
 ('en', ((6, 9), (2, -1))),
 ('en', ((2, 6), (2, -1))),
 ('en', ((4, 5), (2, -1))),
 ('no', ((1, 2), (3, 0))),
 ('no', ((6, 2), (3, 0)))]

In [45]:
Scores = Scores.map(lambda x: x[1]).map(lambda x: ((x[1][0],x[0][0]),x[0][1]*x[1][1])) #((User,Doc),ranking(palabra))
Scores.collect()

[((1, 3), 2),
 ((1, 5), 2),
 ((2, 6), 12),
 ((2, 4), 4),
 ((1, 3), 1),
 ((1, 3), 1),
 ((2, 5), -1),
 ((2, 3), -4),
 ((2, 1), -1),
 ((2, 6), -9),
 ((2, 2), -6),
 ((2, 4), -5),
 ((3, 1), 0),
 ((3, 6), 0)]

In [46]:
Scores = Scores.reduceByKey(lambda x,y: x+y).map(lambda x: x[0]+(x[1],))
Scores.collect()

[(1, 5, 2),
 (2, 4, -1),
 (2, 3, -4),
 (2, 6, 3),
 (1, 3, 4),
 (3, 1, 0),
 (2, 2, -6),
 (2, 5, -1),
 (3, 6, 0),
 (2, 1, -1)]

## EJ NUMERO 2

Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.

In [78]:
chats_detalle = [
    (1, 1,  'rodolf','amigooo, como estas???'),
    (2, 2,  'bianca','hey! holaaa'),
    (3, 3,'juan', 'todo bien y vos?'),
    (4, 3, 'juan', 'No puedo creerlo'),
    (5, 1, 'rodolf', 'mañana salgo YA para allá.ok?')
]

# Los paso a RDDs

rdd_chat = sc.parallelize(chats_detalle)

In [70]:
rdd_chat=rdd_chat.map(lambda x: (x[1],x[3]))
rdd_chat.collect()

[(1, 'amigooo, como estas???'),
 (2, 'hey! holaaa'),
 (3, 'todo bien y vos?'),
 (3, 'No puedo creerlo'),
 (1, 'ma\xc3\xb1ana salgo YA para all\xc3\xa1.ok?')]

In [71]:
rdd_chat=rdd_chat.filter(lambda x: '?' in x[1])
rdd_chat.collect()

[(1, 'amigooo, como estas???'),
 (3, 'todo bien y vos?'),
 (1, 'ma\xc3\xb1ana salgo YA para all\xc3\xa1.ok?')]

In [72]:
rdd_chat.count() #De esta manera cuento las preguntas si considero que puede haber solo una por chat

3

In [77]:
rdd_chat.map(lambda x: (x[0],x[1].count('?'))).reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)


(1, 4)

In [ ]:
#Otra manera de hacerlo (CLASE)

In [82]:
rdd_chat.filter(lambda x: '?' in x[3])\
.map(lambda x: (x[1],x[3].count('?')))\
.reduceByKey(lambda x,y: x+y)\
.reduce(lambda x,y: x if x[1]>y[1] else y)

(1, 4)

## EJ NUMERO 3

UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [99]:
trip = [
    (1, 1,30,  '20171105',500.0),
    (2, 2,40,  '20171105',200.0),
    (3, 3,25,'20160405', 50.0),
    (3, 3,11, '20160411', 1000.0),
    (5, 1,13, '20160406', 800.0)
]

# Los paso a RDDs

rdd_trip = sc.parallelize(trip)

In [111]:
rdd_trip.filter(lambda x: x[3][:6]=='201604')\
.map(lambda x: (x[0],(x[4],1)))\
.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
.map(lambda x: (x[0],(x[1][0]/x[1][1]))).reduce(lambda x,y: x[0] if x[1]>y[1] else y[0])

5

## EJ NUMERO 4

Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares. 

In [112]:
estrellas = [
    (1, 1.12,30,  '20171105'),
    (2, 2.12,40,  '20171105'),
    (3, 3.34,25,'20160405'),
    (4, 3.39,30, '20160411'),
    (5, 1.29,30, '20160406')
]

# Los paso a RDDs

rdd_estrellas = sc.parallelize(estrellas)

In [114]:
# CLASE

In [113]:
rdd_estrellas.map(lambda x:(x[0],(1,x[1],x[2])))\
.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.map(lambda x: ((round(float(x[1][1]/x[1][0]),1),x[1][2]),x[0]))\
.groupByKey()\
.map(lambda x: (x[0],list(x[1])))\
.collect()

[((2.0, 40), [2]), ((3.0, 25), [3]), ((1.0, 30), [1, 5]), ((3.0, 30), [4])]

In [119]:
rdd_estrellas.map(lambda x: ((round(x[1],1),x[2]),x[0])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()

[((2.0, 40), [2]), ((3.0, 25), [3]), ((1.0, 30), [1, 5]), ((3.0, 30), [4])]

In [122]:
rdd_estrellas.map(lambda x: ((round(x[1],1),x[2]),x[0])).groupByKey().mapValues(list).collect()

[((2.0, 40), [2]), ((3.0, 25), [3]), ((1.0, 30), [1, 5]), ((3.0, 30), [4])]

## EJ NUMERO 5

Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas. Ejemplo: [[discounter,introduces,reductions],[percussion,supersonic]...]

In [123]:
import wikipedia as wp
wp.set_lang('en')
articulo1 = wp.page('F-35').content
articulo2 = wp.page('Cajón de rumba').content
rdd = sc.parallelize([articulo1,articulo2])

In [124]:
rdd = rdd.map(lambda x: x.replace("\n","").split(" ")) #tokenizer naive
rdd = rdd.flatMap(lambda x: x).filter(lambda x: len(x) > 7)

In [125]:
rdd = rdd.map(lambda x: x.lower())
rdd = rdd.map(lambda x: (tuple(sorted(x)),x))

In [127]:
rdd = rdd.groupByKey().mapValues(set)


In [129]:
rdd = rdd.filter(lambda x: len(x[1]) > 1)
rdd = rdd.map(lambda x: list(x[1]))
rdd.collect()

[[u'surface-to-air', u'air-to-surface'],
 [u'reduction', u'introduce'],
 [u'supersonic', u'percussion']]

## EJ NUMERO 6

Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)

In [134]:
from random import uniform as uf

In [138]:
rectangulos = [
    (1,2,1,2),
    (2,3,6,8),
    (0,2,1,2),
    (1,2,0,2),
    (1,2,1,3),
    (1,4,1,2),
]

punto = (1.5,1.5)
# Los paso a RDDs

rdd_rec = sc.parallelize(rectangulos)

In [139]:
punto_a_encontrar =punto



def square_contains(square,point=punto_a_encontrar):
    x1, x2 = sorted([square[0],square[1]])
    y1, y2 = sorted([square[2],square[3]])
    return (point[0] >= x1 and point[0] <= x2) and \
            (point[1] >= y1 and point[1] <= y2)
    
#devuelve el area de un cuadrado
def area(square):
    x1, x2 = sorted([square[0],square[1]])
    y1, y2 = sorted([square[2],square[3]])
    return abs(x2-x1)*abs(y2-y1)

In [141]:
rdd_rec = rdd_rec.filter(square_contains)

In [145]:
rdd_rec.min(key=area)

(1, 2, 1, 2)

## EJ NUMERO 7

A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta información escribir un programa en pySpark que permita: a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) 

In [59]:
ventas_diarias = [
    (1,3,'20180605',50.0,15.0),
    (2,3,'20180511',80.0,15.0),
    (3,2,'20170405',90.0,15.0),
    (1,5,'20180305',40.0,15.0),
    (2,4,'20180609',30.0,10.0),
    (3,1,'20180211',90.0,15.0)
]

videojuegos = [
    (1,'jojox',4,3),
    (2,'joj3x',4,3),
    (3,'kel',4,3)    
]



# Los paso a RDDs

rdd_ventas = sc.parallelize(ventas_diarias)
rdd_videojuego = sc.parallelize(videojuegos)

In [60]:
rdd_ventas= rdd_ventas.map(lambda x: (x[0],x[4]))
rdd_videojuego = rdd_videojuego.map(lambda x: (x[0],x[1]))                          

In [61]:

ventas_combinado = rdd_ventas.join(rdd_videojuego)
ventas_combinado.collect()


[(1, (15.0, 'jojox')),
 (1, (15.0, 'jojox')),
 (2, (15.0, 'joj3x')),
 (2, (10.0, 'joj3x')),
 (3, (15.0, 'kel')),
 (3, (15.0, 'kel'))]

In [62]:
rdd_videojuegos=ventas_combinado.reduceByKey(lambda x,y: (x[0]+y[0],x[1])).map(lambda x: (x[0],x[1][1],x[1][0]))
rdd_videojuegos.collect()

[(1, 'jojox', 30.0), (2, 'joj3x', 25.0), (3, 'kel', 30.0)]

 b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)

In [72]:

maximo=rdd_videojuegos.reduce(lambda x,y: x if x[2] > y[2] else y)
rdd_maximo=sc.parallelize(maximo)
rdd_maximo.collect()


[3, 'kel', 30.0]

In [83]:
videoJuegoMasVendido = [ (3,'kel',30.0)]
rddMaximo = sc.parallelize(videoJuegoMasVendido)


In [76]:
rdd_maximo=rdd_maximo.map(lambda x: (x[0],x[2]))


In [78]:
rdd_ventas = sc.parallelize(ventas_diarias)
rdd_ventas=rdd_ventas.map(lambda x: (x[0],(x[1],x[3],x[4])))

rdd_ventas.collect()


[(1, (3, 50.0, 15.0)),
 (2, (3, 80.0, 15.0)),
 (3, (2, 90.0, 15.0)),
 (1, (5, 40.0, 15.0)),
 (2, (4, 30.0, 10.0)),
 (3, (1, 90.0, 15.0))]

In [84]:
nuevo_rdd=rdd_ventas.join(rddMaximo)
nuevo_rdd.collect()


[(3, ((2, 90.0, 15.0), 'kel')), (3, ((1, 90.0, 15.0), 'kel'))]

In [93]:
nuevo_rdd.map(lambda x: (x[1][0][0],float(x[1][0][2]/x[1][0][1]))).reduce(lambda x,y: x if x[1] > y[1] else y)

(1, 0.16666666666666666)

# PANDAS

## EJ NUMERO 8 


Un sitio de E books tiene información sobre los reviews que los usuarios hacen de sus libros en un DataFrame con formato (user_id, book_id, rating, timestamp).

Por otro lado tenemos información en otro DataFrame que bajamos de GoodReads: (book_id, book_name, avg_rating). Podemos suponer que los Ids de los libros son compatibles.

Se pide usar Python Pandas para:

Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame).
Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

In [3]:
import pandas as pd

In [122]:
# Set-up y vista rápida de los dos datasets sintéticos
import pandas as pd

ebooks_df = pd.read_csv('/home/barbara/Documentos/datos/material_datos/ebook.csv')
display(ebooks_df)

goodreads_df = pd.read_csv('/home/barbara/Documentos/datos/material_datos/goodreads.csv')
display(goodreads_df)

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,2,4,2017-05-28 23:38:06
2,3,3,4,2017-05-28 23:38:06
3,4,4,4,2017-05-28 23:38:06
4,5,5,6,2017-05-28 23:38:07
5,6,5,6,2017-05-28 23:38:08
6,1,2,6,2017-05-29 23:38:08
7,1,3,4,2017-05-29 23:38:09
8,1,4,4,2017-05-29 23:38:10
9,1,5,6,2017-05-29 23:38:11


,book_id,book_name,avg_rating
0,1,To Kill a MockingBird,5
1,2,Don Quixote,4
2,3,Lazarillo de Tormes,4
3,4,1984,5
4,5,Fifty Shades of Grey,1
5,6,Los hombres que no amaban a las mujeres,4
6,7,Corazon de tinta,5
7,8,Harry Potter,5


In [123]:
def score(s):
    return s.mean()



In [124]:
ebooks_df.groupby('book_id').agg({'rating':score}).nlargest(5, 'rating')

,rating
book_id,
1,6.2
2,6.0
5,6.0
3,5.5
4,4.0


B) Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

In [126]:
ebooks_df_mean = ebooks_df.groupby('book_id').agg({'rating':score})
ebooks_df_mean


,rating
book_id,
1,6.2
2,6.0
3,5.5
4,4.0
5,6.0


In [127]:
df_total = goodreads_df.join(ebooks_df_mean, on='book_id',how='inner')
df_total

,book_id,book_name,avg_rating,rating
0,1,To Kill a MockingBird,5,6.2
1,2,Don Quixote,4,6.0
2,3,Lazarillo de Tormes,4,5.5
3,4,1984,5,4.0
4,5,Fifty Shades of Grey,1,6.0


In [130]:
df_total[(df_total.rating/df_total.avg_rating - 1.0).abs() > 0.2].book_name.tolist()

['To Kill a MockingBird',
 'Don Quixote',
 'Lazarillo de Tormes',
 'Fifty Shades of Grey']

## EJ NUMERO 9

Tenemos un dataframe con la información de distintas playlists armadas por usuarios con el formato (playlist, song_id, description).

A su vez, contamos con un dataframe de canciones que contiene (song_id, singer, year, lenght, genres). Se pide generar un programa en Pandas que indique para cada playlist cual es el cantante predominante (con mas canciones incluidas dentro de esa lista)



In [156]:
df_playlist = pd.read_csv('/home/barbara/Documentos/datos/material_datos/playlist.csv')
display(df_playlist)

df_canciones = pd.read_csv('/home/barbara/Documentos/datos/material_datos/canciones.csv')
display(df_canciones)


,playlist,song_id,description
0,Los Ochenta,1,Muy buena
1,Los Ochenta,2,Pesima
2,Los Ochenta,3,Re divertida!
3,Los Ochenta,4,Excelente
4,Los Noventa,5,Excelente
5,Los Noventa,6,Excelente
6,Los Noventa,7,Excelente
7,Los Noventa,8,Excelente


,song_id,singer,year,length,genres
0,1,Firulo,1986,3m,rock
1,2,Firulo,1986,3m,rock
2,3,Pipi,1987,3m,rock
3,4,Kratos,1985,3m,rock
4,5,Pinoccio,1996,3m,rock
5,6,Batman,1996,3m,rock
6,7,Pinoccio,1997,3m,rock
7,8,Fast,1992,3m,rock


In [147]:
df_playlist['cantidad']=1

In [153]:
ff=df_playlist.merge(df_canciones, how='inner').groupby(['playlist','singer']).agg({'cantidad':'sum'}).reset_index()
ff

,playlist,singer,cantidad
0,Los Noventa,Batman,1
1,Los Noventa,Fast,1
2,Los Noventa,Pinoccio,2
3,Los Ochenta,Firulo,2
4,Los Ochenta,Kratos,1
5,Los Ochenta,Pipi,1


In [155]:
ff.groupby('playlist').max()

,singer,cantidad
playlist,,
Los Noventa,Pinoccio,2
Los Ochenta,Pipi,2


In [ ]:
#Ahora de la manera que encontré en la web.

In [157]:
df_playlist.merge(df_canciones, how='left').groupby('playlist')['singer'].agg(lambda x:x.mode())

playlist
Los Noventa    Pinoccio
Los Ochenta      Firulo
Name: singer, dtype: object

## EJERCICIO NUMERO 10

La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos cuando los padres registran a sus hijos en el registro civil a partir de una encuesta.

Esa información se encuentra disponible para su análisis en un csv con el siguiente formato (dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer, id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea.

Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual).

Se pide usar Pandas para:

Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales.

Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.

In [56]:
registro  =  [('01', '02', '2001', 5.1, 54, 34, 2),
         ('02', '02', '2001', 5.2, 54, 35, 2),
         ('03', '03', '2001', 5.3, 54, 34, 2),
         ('04', '04', '2001', 5.4, 54, 36, 2),
         ('21', '10', '2016', 3.4, 54, 34, 2),
         ('22', '10', '2016', 5.4, 54, 34, 1),
         ('05', '10', '2016', 5.4, 54, 34, 1),
         ('01', '10', '2016', 1.4, 54, 31, 2),
         ('06', '10', '2016', 5.4, 54, 32, 1),
         ('05', '10', '2017', 4.4, 54, 34, 2),
         ('15', '10', '2017', 3.4, 54, 31, 2),
         ('16', '10', '2017', 5.8, 54, 31, 1),
         ('13', '10', '2017', 5.9, 54, 34, 2),
         ('18', '10', '2017', 5.4, 54, 23, 1),
         ('21', '10', '2017', 4.4, 54, 23, 2),
         ('01', '10', '2017', 5.4, 54, 34, 2)]

hospitales = [(23, 3.4),
       (31, 1.3),
       (32, 0.3),
       (34, 2.3),
       (35, 10.3),
       (36, 5.7)]

registro = pd.DataFrame(registro, columns=["dia", "mes", "anio", "peso", "altura", "hospital", "tipo"])
hospitales = pd.DataFrame(hospitales, columns=["hospital", "promedio"])

In [33]:
registro['Oct2017']=(registro['mes']=='10')&(registro['anio']=='2017')

In [5]:
registro=registro.loc[(registro['Oct2017']==True),['hospital']]
registro

,hospital
9,34
10,31
11,31
12,34
13,23
14,23
15,34


In [6]:
merge=registro.merge(hospitales,left_on="hospital",right_on= "hospital",how='inner')
merge

,hospital,promedio
0,34,2.3
1,34,2.3
2,34,2.3
3,31,1.3
4,31,1.3
5,23,3.4
6,23,3.4


In [7]:
merge['cantNacimientos']=1

In [8]:
merge=merge.groupby('hospital').agg({'promedio':'first','cantNacimientos':'sum'}).reset_index()

In [14]:
merge.loc[(merge['promedio']<merge['cantNacimientos']),('hospital')]

1    31
2    34
Name: hospital, dtype: int64

Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.

In [ ]:
#COmo lo encontré en la web

In [60]:
octubre = registro[registro['mes'] == '10']
porc_cesarea = octubre.groupby('anio').tipo.agg(lambda x: (x==2).mean())

porc_cesarea

anio
2016    0.400000
2017    0.714286
Name: tipo, dtype: float64

In [62]:

if porc_cesarea['2017'] > porc_cesarea['2016']: 
    print("Se incrementó el % de cesáreas")
else: 
    print("No se incrementó el % de cesáreas")

Se incrementó el % de cesáreas


In [ ]:
#Otra forma

In [65]:
registro['con_cesarea'] = registro['tipo'] == 2
merged_a = registro.loc[(registro['mes']=='10') & ((registro['anio'] == '2016') | (registro['anio'] == '2017')), ['con_cesarea', 'anio']].groupby('anio').mean()
merged_a


,con_cesarea
anio,
2016,0.400000
2017,0.714286


In [68]:
merged_a['con_cesarea'][1] > merged_a['con_cesarea'][0]

True

## EJERCICIO NUMERO 11

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución.

Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respu esta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes situaciones:

a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).

b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución).

c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.



In [119]:
CASO_ABIERTO = 1
CASO_RESUELTO = 2
CASO_CERRADO = 3

BATMAN_NO_PARTICIPA = 0
BATMAN_PARTICIPA = 1

df_gcpd = pd.read_csv('/home/barbara/Documentos/datos/material_datos/GCPD.csv')
display(df_gcpd)

df_gordon = pd.read_csv('/home/barbara/Documentos/datos/material_datos/gordon.csv')
display(df_gordon)

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2016-05-03,1,Mataron a uno,1,Arseny,100,100
1,2016-05-03,2,Mataron al Guason!!,2,Robo,100,100
2,2017-07-03,3,Incendio,1,Robo,100,100
3,2017-07-03,4,Robaron un banco,3,Incendio,100,100
4,2017-08-03,5,Mataron a uno,2,Incendio,100,100
5,2017-08-03,6,Mataron a uno,1,Arseny,100,100
6,2017-10-03,7,Mataron a uno,2,Robo,100,100
7,2017-10-03,8,Mataron a uno,3,Incendio,100,100
8,2017-11-03,9,Mataron a uno,2,Arseny,100,100
9,2017-12-03,10,Mataron a uno,1,Robo,100,10


,id_caso,respuesta
0,1,1
1,2,0
2,3,0
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1
9,10,0


a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando TODOS los casos)

In [98]:
df_gcpd.groupby(['categoria'])['estado_caso'].agg(lambda x: (x == CASO_RESUELTO).mean()).rename('tasa de resolución')

categoria
Arseny      0.333333
Incendio    0.333333
Robo        0.500000
Name: tasa de resolución, dtype: float64

In [ ]:
Ahora considero los que NO participo batman

a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).

In [109]:
nuevo_gordon=df_gordon.loc[df_gordon['respuesta'] == 0,:]

In [110]:
total=df_gcpd.merge(nuevo_gordon,left_on='id_caso',right_on='id_caso',how='inner')

In [111]:
total.groupby('categoria')['estado_caso'].agg(lambda x: (x==2).mean())

categoria
Arseny      0.000000
Incendio    0.000000
Robo        0.333333
Name: estado_caso, dtype: float64

b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución).

In [112]:
nuevo_gordon=df_gordon.loc[df_gordon['respuesta'] == 1,:]

In [113]:
total=df_gcpd.merge(nuevo_gordon,left_on='id_caso',right_on='id_caso',how='inner')

In [150]:
total.estado_caso.agg(lambda x: (x==CASO_RESUELTO).mean())

0.75

In [148]:
total.estado_caso.apply(lambda x: x==CASO_RESUELTO).mean()

0.75

c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.


In [126]:
df_pasado=df_gcpd.loc[(df_gcpd['fecha'] >=  "2017-01-01") & (df_gcpd['fecha'] <=  "2018-01-01") ,:]

In [134]:
total2=df_pasado.merge(df_gordon,left_on='id_caso',right_on='id_caso',how='inner')
total2


,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta
0,2017-07-03,3,Incendio,1,Robo,100,100,0
1,2017-07-03,4,Robaron un banco,3,Incendio,100,100,0
2,2017-08-03,5,Mataron a uno,2,Incendio,100,100,1
3,2017-08-03,6,Mataron a uno,1,Arseny,100,100,0
4,2017-10-03,7,Mataron a uno,2,Robo,100,100,1
5,2017-10-03,8,Mataron a uno,3,Incendio,100,100,0
6,2017-11-03,9,Mataron a uno,2,Arseny,100,100,1
7,2017-12-03,10,Mataron a uno,1,Robo,100,10,0


In [147]:
total2.respuesta.agg( lambda x: (x==1).mean())

0.375

In [152]:
total2['fechaOK']=pd.to_datetime(total2['fecha'])
#total2['anio']=casos['fecha_ok'].map(lambda x:x.year)
total2['mes']=total2['fechaOK'].map(lambda x:x.month)

In [153]:
total2.groupby('mes')['respuesta'].agg( lambda x: (x==1).mean())

mes
7     0.0
8     0.5
10    0.5
11    1.0
12    0.0
Name: respuesta, dtype: float64

## EJ NUMERO 12

El GCPD recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta informacióñ se encuentra guarada en un archivo con el siguiente formato:

(fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)

Los posibels estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolucion.

Las fechas se encuentran en el formato YYYY-MM-DD.

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuales casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respeusta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.

b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

Resolver ambas consultas utilizando Pandas.

In [216]:
casos = [(1, '20171201', 'robo', 1, 'robo'),
         (3, '20181103', 'pelea', 2, 'pelea'),
         (12, '20180703', 'pelea', 1, 'pelea'),
         (13, '20180713', 'pelea', 2, 'pelea'),
         (14, '20180721', 'pelea', 1, 'pelea'),
        (15, '20180811', 'pelea', 2, 'pelea'),
        (16, '20180813', 'pelea', 2, 'pelea'),
        (17, '20180813', 'pelea', 1, 'pelea'),
         (4, '20180404', 'robo', 2, 'robo'),
         (9, '20180414', 'robo', 1, 'robo'),
        (2, '20180911', 'robo', 2, 'robo'),
        (10, '20180912', 'robo', 1, 'robo'),
        (11, '20180913', 'robo', 1, 'robo'),
         (8, '20180604', 'robo', 1, 'robo'),
         (5, '20161204', 'asesinato', 1, 'asesinato'),
         (6, '20180510', 'robo', 2, 'robo'),
         (7, '20171211', 'bandalismo', 2, 'bandalismo')]

respuesta = [(1, 0),
            (3, 1),
            (4, 1),
            (5, 0),
            (7, 1),
            (8, 1)]

#rdd_casos = sc.parallelize(casos)
#rdd_respuesta = sc.parallelize(respuesta)
casos = pd.DataFrame(casos, columns=["id_casos","fecha", "descripcion", "estado", "categoria"])
respuesta = pd.DataFrame(respuesta, columns=["id_casos", "respuesta"])

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.


In [217]:
casos['ultimoTrimestre'] = (casos['fecha'] >= '20180901') & (casos['fecha'] <= '20181231')

In [219]:
casos.loc[(casos['fecha'] >= '20180601') & (casos['fecha'] <= '20181231') ,:].groupby(['ultimoTrimestre','categoria']).estado.agg(lambda x: (x==2).mean())


ultimoTrimestre  categoria
False            pelea        0.500000
                 robo         0.000000
True             pelea        1.000000
                 robo         0.333333
Name: estado, dtype: float64

In [207]:
# Otra forma (usando un csv)
df_gcpd['fecha'] = pd.to_datetime(df_gcpd['fecha'],format='%Y-%m-%d')


df_gcpd['trimestre'] = df_gcpd.fecha.dt.month % 4
df_trimestres = df_gcpd[df_gcpd.trimestre.isin([3,2])]
df_tasa_por_trimestre = df_trimestres.pivot_table(columns='trimestre',index='categoria',aggfunc=lambda x: (x==CASO_RESUELTO).mean(),values='estado_caso').fillna(0)
df_tasa_por_trimestre[df_tasa_por_trimestre[3] > df_tasa_por_trimestre[2]].index.values.tolist()

['Arseny']

b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

In [223]:
DELITOS_CONTRA_LA_PROPIEDAD = set(['Incendio', 'Robo', 'Hurto', 'Robo de Vehículo']) 
df = df_gcpd.merge(df_gordon,how='left',on='id_caso')
df = df[df.categoria.isin(DELITOS_CONTRA_LA_PROPIEDAD)] 
df.groupby('categoria')['respuesta'].apply(lambda x: (x==BATMAN_PARTICIPA).mean()).rename('tasa de respuesta')

categoria
Incendio    0.333333
Robo        0.250000
Name: tasa de respuesta, dtype: float64

## PARCIAL FALLIDO SPARK, Y PANDAS.

### EJ NUMERO 12

Spark: Tenemos información sobre recetas en 3 RDD de Spark.
    Recetas: (id_receta, nombre, tiempo_preparación, dificultad)
    Ingredientes: (id_ingrediente, nombre)
    Ingredientes por receta: (id_receta, id_ingrediente, cantidad)
        
Se pide:
    a) obtener el nombre de todas las recetas que tenggan Cordero.
    b) Calcular la cantidad total de cada ingrediente si queremos hacer todos las recetas con Cordera que sean fáciles.
    

In [2]:
Recetas = [
    (1, "brownies locos", 40, "facil"),
    (2, "brownies vainilla", 30, "re facil"),
    (3, "salsa de cordero", 120, "facil"),
    (4, "cordero mechado", 180, "dificil"),
    (5, "picada de cordero", 30, "medio"),
    (6, "cordero al roquefort", 80, "facil"),
]

Ingredientes = [
    (3,"cordero"),
    (1,"roquefort"),
    (2,"harina"),
    (4,"queso"),
    (5,"tomate"),
    (6,"huevo")
]

IngredientesPorReceta = [
    (1,2,1),
    (3,3,2),
    (4,3,0.5),
    (5,3,0.3),
    (6,3,1),
    (3,5,3),
    (6,4,4)    
]

# Los paso a RDDs

rdd_recetas = sc.parallelize(Recetas)
rdd_ing = sc.parallelize(Ingredientes)
rdd_ingxrecetas = sc.parallelize(IngredientesPorReceta)

a) obtener el nombre de todas las recetas que tenggan Cordero

In [10]:
#Primero debo saber que ingrediente tiene cada receta, 
#dspues buscar en ellos el cordero, si pertenece entonces me quedo con esa receta.

# vamos a mergear los 3 rdd. (ya que necesito los ingredientes x receta (asi obtengo el codigo) y los ingredientes(para obtener el nombre))

#Primero mapeo recetas
recetas=rdd_recetas.map(lambda x: (x[0],(x[1],x[3]))) #(id_receta,(nombre,dificultad))
ingxr=rdd_ingxrecetas.map(lambda x: (x[0],(x[1],x[2])))

recetaconingrediente=recetas.join(ingxr)


In [4]:
rinMapped=recetaconingrediente.map(lambda x: (x[1][1][0],(x[0],x[1][0][0],x[1][0][1],x[1][1][1])))
final=rinMapped.join(rdd_ing)

In [5]:
ffiltrado=final.filter(lambda x: x[1][1] == 'cordero')
ffiltrado.collect()

[(3, ((4, 'cordero mechado', 'dificil', 0.5), 'cordero')),
 (3, ((5, 'picada de cordero', 'medio', 0.3), 'cordero')),
 (3, ((6, 'cordero al roquefort', 'facil', 1), 'cordero')),
 (3, ((3, 'salsa de cordero', 'facil', 2), 'cordero'))]

In [236]:
ffiltrado.map(lambda x: x[1][0][1]).collect()

['cordero mechado',
 'picada de cordero',
 'cordero al roquefort',
 'salsa de cordero']

 b) Calcular la cantidad total de cada ingrediente si queremos hacer todos las recetas con Cordera que sean fáciles.

In [6]:
#Uso final ( merge de los 3)
final.collect()

[(2, ((1, 'brownies locos', 'facil', 1), 'harina')),
 (3, ((4, 'cordero mechado', 'dificil', 0.5), 'cordero')),
 (3, ((5, 'picada de cordero', 'medio', 0.3), 'cordero')),
 (3, ((6, 'cordero al roquefort', 'facil', 1), 'cordero')),
 (3, ((3, 'salsa de cordero', 'facil', 2), 'cordero')),
 (4, ((6, 'cordero al roquefort', 'facil', 4), 'queso')),
 (5, ((3, 'salsa de cordero', 'facil', 3), 'tomate'))]

In [7]:
ffiltrado2=final.filter(lambda x: x[1][0][2] == 'facil')
ffiltrado2.collect()

[(2, ((1, 'brownies locos', 'facil', 1), 'harina')),
 (3, ((6, 'cordero al roquefort', 'facil', 1), 'cordero')),
 (3, ((3, 'salsa de cordero', 'facil', 2), 'cordero')),
 (4, ((6, 'cordero al roquefort', 'facil', 4), 'queso')),
 (5, ((3, 'salsa de cordero', 'facil', 3), 'tomate'))]

In [11]:
ffiltrado2=ffiltrado2.filter(lambda x: x[1][1] == 'cordero')
ingredientesPedidos=ffiltrado2.join(ingxr)

In [13]:
ffiltrado2.collect()


[(3, ((6, 'cordero al roquefort', 'facil', 1), 'cordero')),
 (3, ((3, 'salsa de cordero', 'facil', 2), 'cordero'))]

In [16]:
ingredientesPedidos.collect()


[(3, (((6, 'cordero al roquefort', 'facil', 1), 'cordero'), (3, 2))),
 (3, (((6, 'cordero al roquefort', 'facil', 1), 'cordero'), (5, 3))),
 (3, (((3, 'salsa de cordero', 'facil', 2), 'cordero'), (3, 2))),
 (3, (((3, 'salsa de cordero', 'facil', 2), 'cordero'), (5, 3)))]

In [21]:
ingredientesPedidos.map(lambda x: (x[1][1][0],x[1][1][1])).reduceByKey(lambda x,y: (x + y)).collect()

[(3, 4), (5, 6)]

## EJ NUMERO 13

Dada la exitosa convocatoria de los juegos olímpicos de la juventud por parte del público, sus organizadores realizan distintos análisis para planificar las jornadas finales del certamen. Es por ello que cuentan con información en los siuientes archivos

eventos.csv (id_evento, fecha, id_location, nombre_evento, id_categoria_deportiva_deportiva, cantidad_espectadores)
location.csv (id_locacion, nombre, capacidad, capacidad_extendida, sede , latitud, longitud)
categorias_deportivas.csv(id_categoria_deportiva, nombre, año_de_adopcion)

El primer archivo cuenta con información de los eventos, indicando la fecha en formato "YYYY-mm-dd hh:mm:ss", el lugar donde ocurrió (id_locacion) y la cantidad de espectadores que asistieron. Ademñas se aporta información sobre la categoría deportiva a la cual pertenece el evento.
Por otro lado se tiene información sobre las distintas locaciones en la sedes del certamen en las que ocurrieron los eventos. Contamos con información de sus capacidad extendida (cuantos asientos extras se pueden brindar sobre la capacidad de la locación)

Se desea obtener:

a)Nombre de la sede que acumuló la mayor cantidad de espectadores en eventos durante el certamen del 14 al 15 de octubre inclusive. Esto es de vital importancia para distribuir el mercchadising oficial del evento, para las fechas finales .

b) Nombre del evento y nombre de la cateoria deportiva de auqelllos eventos cuya cantidad de espectadores superó la capacidad de la locación, más allá de la capacidad extendida. Esto es de vital importancia para detectar problemas de seguridad o si es necesario realizar algun cambio de locación. 




In [74]:
eventos = [(1, '2017-10-14 16:30:45', 1 , "tennis de mesa", 1,400),
           (2, '2017-10-14 16:40:45', 1 , "natacion olimpica", 2,1000),
           (3, '2017-10-14 16:30:45', 1 , "hockey sobre hielo", 3,1300),
           (4, '2017-12-15 16:30:45', 2 , "natacion competencia", 2,1500),
           (5, '2017-10-15 16:30:45', 2 , "hockey final", 3,1800),
           (6, '2017-10-16 16:30:45', 2 , "tennis final", 1,1600)
         ]

locacion = [(1, "rolando carreras",600,300,"sede azul",40,40),
            (2, "full training",1000,600,"sede rosa",40,40)
            ]

categoriasDeportivas = [(1,"tennis",1990),
                        (2,"natacion",1995),
                        (3,"hockey",1993)
                       ]

#rdd_casos = sc.parallelize(casos)
#rdd_respuesta = sc.parallelize(respuesta)
rdd_eventos = pd.DataFrame(eventos, columns=["id_evento", "fecha", "id_location", "nombre_evento", "id_categoria_deportiva", "cantidad_espectadores"])
rdd_locacion = pd.DataFrame(locacion, columns=["id_locacion", "nombre", "capacidad", "capacidad_extendida", "sede" , "latitud", "longitud"])
rdd_cat_dep = pd.DataFrame(categoriasDeportivas, columns= ["id_categoria_deportiva", "nombre", "año_de_adopcion"])

Nombre de la sede que acumuló la mayor cantidad de espectadores en eventos durante el certamen del 14 al 15 de octubre inclusive. Esto es de vital importancia para distribuir el mercchadising oficial del evento, para las fechas finales .

In [45]:
rdd_eventos['fechaOK']=pd.to_datetime(rdd_eventos['fecha'])

In [46]:
rdd_eventos['anio']=rdd_eventos['fechaOK'].dt.year
rdd_eventos['mes']=rdd_eventos['fechaOK'].dt.month
rdd_eventos['dia']=rdd_eventos['fechaOK'].dt.day
rdd_eventos

,id_evento,fecha,id_location,nombre_evento,id_categoria_deportiva_deportiva,cantidad_espectadores,fechaOK,anio,mes,dia
0,1,2017-10-14 16:30:45,1,tennis de mesa,1,400,2017-10-14 16:30:45,2017,10,14
1,2,2017-10-14 16:40:45,1,natacion olimpica,2,1000,2017-10-14 16:40:45,2017,10,14
2,3,2017-10-14 16:30:45,1,hockey sobre hielo,3,1300,2017-10-14 16:30:45,2017,10,14
3,4,2017-12-15 16:30:45,2,natacion competencia,2,1500,2017-12-15 16:30:45,2017,12,15
4,5,2017-10-15 16:30:45,2,hockey final,3,1800,2017-10-15 16:30:45,2017,10,15
5,6,2017-10-16 16:30:45,2,tennis final,1,1600,2017-10-16 16:30:45,2017,10,16


In [58]:
listoParaMerge = rdd_eventos.loc[(rdd_eventos['mes']==10) & (13<rdd_eventos['dia']) & (rdd_eventos['dia']<16),['id_location','cantidad_espectadores']]


In [61]:
final=listoParaMerge.merge(rdd_locacion,left_on='id_location',right_on='id_locacion',how='inner')

In [67]:
final.max().nombre

'rolando carreras'

 Nombre del evento y nombre de la cateoria deportiva de auqelllos eventos cuya cantidad de espectadores superó la capacidad de la locación más allá de la capacidad extendida. Esto es de vital importancia para detectar problemas de seguridad o si es necesario realizar algun cambio de locación.

In [78]:
medio=rdd_eventos.merge(rdd_locacion,left_on='id_location',right_on='id_locacion',how='inner')

In [79]:
medio2=medio.loc[(medio['cantidad_espectadores']>(medio['capacidad']+medio['capacidad_extendida'])),['id_categoria_deportiva',"nombre_evento"]]

In [80]:
final=medio2.merge(rdd_cat_dep,left_on='id_categoria_deportiva',right_on='id_categoria_deportiva',how='inner')

In [82]:
final.loc[:,['nombre',"nombre_evento"]]

,nombre,nombre_evento
0,natacion,natacion olimpica
1,hockey,hockey sobre hielo
2,hockey,hockey final
